## Load data yang dibutuhkan

Untuk data gizi pakai data konsumsi di tahun 2012 dulu

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

## ENTAH KENAPA TAPI KAYANYA WERT DAN WEINDNYA KEBALIK !!! 

In [2]:
# load data susenas 2012 modul konsumsi
sus12_kons_amt = pd.read_csv('./susenas12/sn12_m41p1.csv')
sus12_kons_gizi = pd.read_csv('./susenas12/sn12_m41p2.csv')
sus12_kons_gabungan = pd.read_csv('./susenas12/sn12_m43.csv')

# load data susenas 2013 modul konsumsi
sus13_con = pd.read_csv('./susenas13/sn13_m41.csv')

In [3]:
# load data susenas modul individu dan rumah tangga jika diperlukan
sus12_indv = pd.read_csv('./susenas12/sn12_ki.csv')
sus12_rt = pd.read_csv('./susenas12/sn12_krt.csv')

In [4]:
sus12_kons_gizi.head()

,URUT,B1R1,B1R2,B1R5,B1R8,B2R1,KODE,KLP,KALORI,PROTEIN,LEMAK,KARBO,WERT,WEIND
0,110112500101601,11,1,1,1,4,1,0,21732.0,508.5,87.00,4653.0,45.876255,183.50502
1,110112500101601,11,1,1,1,4,2,1,21732.0,508.5,87.00,4653.0,45.876255,183.50502
2,110112500101601,11,1,1,1,4,20,0,2269.0,369.3,77.04,0.9,45.876255,183.50502
3,110112500101601,11,1,1,1,4,22,20,904.0,136.0,36.00,0.0,45.876255,183.50502
4,110112500101601,11,1,1,1,4,34,20,904.0,136.0,36.00,0.0,45.876255,183.50502


In [5]:
sus12_kons_gabungan.head()

,URUT,B1R1,B1R2,B1R5,B1R8,B2R1,FOOD,NFOOD,EXPEND,KAPITA,KALORI,WERT,WEIND,EXP_CAP,KABU
0,110112500101601,11,1,1,1,4,8.168571e+05,5.570000e+05,1.373857e+06,3.434643e+05,1277.633929,45.876255,183.505020,3.434643e+05,1101
1,110112500101602,11,1,1,2,4,1.334571e+06,9.466667e+05,2.281238e+06,5.703095e+05,1793.693929,55.252186,221.008743,5.703095e+05,1101
2,110112500101603,11,1,1,3,4,2.901429e+06,2.321667e+06,5.223095e+06,1.305774e+06,2593.590357,20.006414,80.025658,1.305774e+06,1101
3,110112500101604,11,1,1,4,6,1.394571e+06,1.016000e+06,2.410571e+06,4.017619e+05,1430.819048,42.398369,254.390213,4.017619e+05,1101
4,110112500101605,11,1,1,5,4,1.588286e+06,1.212000e+06,2.800286e+06,7.000714e+05,1482.632143,43.420265,173.681061,7.000714e+05,1101


### PERBAIKI KOLOM WERT DAN WEIND YANG TERTUKAR

entah kenapa kedua kolom ini sepertinya nilainya tertukar. Might as well correct them

In [6]:
tukar = {"WERT":"WEIND","WEIND":"WERT"}
sus12_kons_amt = sus12_kons_amt.rename(columns=tukar)
sus12_kons_gabungan = sus12_kons_gabungan.rename(columns=tukar)
sus12_kons_gizi = sus12_kons_gizi.rename(columns=tukar)

### Validasi metode perhitungan angka indikator gizi

Data konsumsi kalori per kapita untuk tiap rumah tangga telah tersedia di bagian susenas konsumsi blok 43, namun data untuk konsumsi protein,lemak, dll. tidak tersedia di dataset tersebut. Maka dapat menggunakan data konsumsi blok 41 dengan cara aggregasi tiap rumah tangga. Untuk memastikan bahwa metode aggregasi dan perhitungan konsumsi indikator gizi telah sesuai, di bawah ini dilakukan perbandingan antara perhitungan kalori per hari/kapita untuk 1 rumah tangga dari modul konsumsi blok 41 dengan data dari blok 43

In [7]:
# Kode dari group makanan yang akan dimasukkan ke dalam perhitungan
# untuk mencegah adanya duplikasi perhitungan antara grup dan tiap item individual ,
# misal kode 1 = total untuk padi-padian, kode 2 = beras tidak dimasukkan karena merupakan bagian dari kelompok 1
food_code = [1, 10, 20, 53, 71, 85, 115, 127, 151, 158, 167, 181, 191]

# cek data kalori di modul konsumsi 
# ambil sampel satu rumah tangga untuk dihitung kalorinya
sample_rt = sus12_kons_gizi.iloc[1,0]
data_sample = sus12_kons_gizi.loc[(sus12_kons_gizi['KODE'].isin(food_code)) & (sus12_kons_gizi['URUT'] == sample_rt)]

data_sample_grouped = data_sample.groupby('URUT').agg(kalori = ("KALORI", 'sum')) # jumlah kalori per minggu untuk 1 rt
print(data_sample_grouped/7/4) # jumlah kalori per hari/kapita , dibagi 4 karena anggota keluarganya ada 4

# bandingkan dengan data di konsumsi gabungan
comparison = sus12_kons_gabungan.loc[sus12_kons_gabungan['URUT'] == sample_rt, 'KALORI']
comparison

                      kalori
URUT                        
110112500101601  1277.633929


0    1277.633929
Name: KALORI, dtype: float64

Angkanya sama, maka metode ini bisa digunakan

In [8]:
data_sample

,URUT,B1R1,B1R2,B1R5,B1R8,B2R1,KODE,KLP,KALORI,PROTEIN,LEMAK,KARBO,WEIND,WERT
0,110112500101601,11,1,1,1,4,1,0,21732.00,508.500,87.000,4653.000,45.876255,183.50502
2,110112500101601,11,1,1,1,4,20,0,2269.00,369.300,77.040,0.900,45.876255,183.50502
6,110112500101601,11,1,1,1,4,71,0,685.30,55.180,48.060,3.115,45.876255,183.50502
8,110112500101601,11,1,1,1,4,85,0,253.80,14.520,3.030,37.935,45.876255,183.50502
13,110112500101601,11,1,1,1,4,115,0,904.00,50.600,85.600,42.200,45.876255,183.50502
15,110112500101601,11,1,1,1,4,127,0,928.80,11.180,2.580,242.520,45.876255,183.50502
17,110112500101601,11,1,1,1,4,151,0,4943.50,12.650,316.120,52.100,45.876255,183.50502
20,110112500101601,11,1,1,1,4,158,0,3992.00,17.400,1.300,1009.000,45.876255,183.50502
23,110112500101601,11,1,1,1,4,167,0,65.35,3.352,1.003,14.442,45.876255,183.50502
27,110112500101601,11,1,1,1,4,191,0,0.00,0.000,0.000,0.000,45.876255,183.50502


# 1. Perhitungan AKG per kapita/hari (kalori,lemak,protein,karbohidrat)

Pertama buat aggregasi untuk tiap rumah tangga dengan groupby nomor urut rumah tangga

In [9]:
# filter berdasarkan group / kelompok makanan
gizi_weekly= sus12_kons_gizi.loc[(sus12_kons_gizi['KODE'].isin(food_code))]

# grouping per rumah tangga
# WEIND, WERT , dan B2R1 aggregasi mean karena nilainya untuk tiap rumah tangga sama
gizi_grouped_weekly = gizi_weekly.groupby('URUT').agg(kalori = ('KALORI', 'sum'),
                                                     protein = ('PROTEIN', 'sum'),
                                                     lemak = ('LEMAK', 'sum'),
                                                     karbohidrat = ('KARBO', 'sum'),
                                                     WERT = ('WERT', 'mean'),
                                                     WEIND = ('WEIND', 'mean'),
                                                     jumlah_art = ('B2R1', 'mean')). reset_index()
gizi_grouped_weekly


,URUT,kalori,protein,lemak,karbohidrat,WERT,WEIND,jumlah_art
0,110112500101601,35773.75,1042.682,621.733,6055.212,183.505020,45.876255,4
1,110112500101602,50223.43,1378.968,1060.152,7904.175,221.008743,55.252186,4
2,110112500101603,72620.53,2102.589,1941.695,10785.024,80.025658,20.006414,4
3,110112500101604,60094.40,1664.150,1014.770,10617.510,254.390213,42.398369,6
4,110112500101605,41513.70,1270.500,921.645,6406.505,173.681061,43.420265,4
...,...,...,...,...,...,...,...,...
286108,947122001103806,48365.80,2037.228,1805.240,6255.670,2190.646820,438.129364,5
286109,947122001103807,68975.98,3158.530,2566.832,7207.554,1605.079956,321.015991,5
286110,947122001103808,30318.58,1376.580,998.652,3922.144,649.540222,324.770111,2
286111,947122001103809,86519.80,2589.050,2170.900,13619.380,98.785879,14.112268,7


## AKG per hari per kapita untuk tiap rumah tangga

In [10]:
# list indikator
indikator = ['kalori', 'protein', 'lemak', 'karbohidrat']

# indikator kecukupan gizi per kapita per hari
gizi_grouped_daily = gizi_grouped_weekly.apply(lambda x: x/7/gizi_grouped_weekly['jumlah_art'] 
                                               if x.name in indikator else x, axis=0)
gizi_grouped_daily

,URUT,kalori,protein,lemak,karbohidrat,WERT,WEIND,jumlah_art
0,110112500101601,1277.633929,37.238643,22.204750,216.257571,183.505020,45.876255,4
1,110112500101602,1793.693929,49.248857,37.862571,282.291964,221.008743,55.252186,4
2,110112500101603,2593.590357,75.092464,69.346250,385.179429,80.025658,20.006414,4
3,110112500101604,1430.819048,39.622619,24.161190,252.797857,254.390213,42.398369,6
4,110112500101605,1482.632143,45.375000,32.915893,228.803750,173.681061,43.420265,4
...,...,...,...,...,...,...,...,...
286108,947122001103806,1381.880000,58.206514,51.578286,178.733429,2190.646820,438.129364,5
286109,947122001103807,1970.742286,90.243714,73.338057,205.930114,1605.079956,321.015991,5
286110,947122001103808,2165.612857,98.327143,71.332286,280.153143,649.540222,324.770111,2
286111,947122001103809,1765.710204,52.837755,44.304082,277.946531,98.785879,14.112268,7


### Rata-Rata AKG tingkat nasional

Perhitungan rata-rata konsumsi tiap indikator gizi secara nasional

In [21]:
# Creating Lambda Function for Weighted Average and Weighted Sum
wt_mean = lambda x: np.average(x, weights=df_sliced.loc[x.index, "WERT"])
wt_sum = lambda x: np.sum(x, weights=df_sliced.loc[x.index, "WERT"])

# hitung rata-rata dengan mempertimbangkan weight rumah tangga
# AKG per kapita untuk tiap rumah tangga berbeda bobotnya sehingga perlu diperhitungkan juga
# weight tiap rumah tangga

AKG_weighted = gizi_grouped_daily.apply(lambda x: np.average(x, weights=gizi_grouped_daily.loc[x.index, "WERT"])
                                         if x.name in indikator else x, axis=0)[indikator].mean()

In [22]:
AKG_weighted

kalori         1858.036745
protein          53.612277
lemak            43.534397
karbohidrat     285.826488
dtype: float64

bagaimana jika dihitung tanpa weight?

In [23]:
kalori_mean = gizi_grouped_daily['kalori'].mean()
kalori_mean

1981.511722764513

#### proporsi penduduk (semua golongan usia) yang asupan kalorinya kurang dari batas 2150


In [24]:
iseng = gizi_grouped_daily.loc[gizi_grouped_daily['kalori'] < 2150 ]

In [25]:
int(iseng['WERT'].sum())/int(gizi_grouped_daily['WERT'].sum()) * 100

75.82396365429814

# 2. Distribusi per daerah dan kota/desa serta tingkat pendapatan 

### A. Cek apakah data RT bisa di collapse ke level kabupaten/kota maupun provinsi

In [26]:
# cek jumlah data rt di dataset akg harian
jumlah_data = '{0:,}'.format(len(set(gizi_grouped_daily.URUT)))
jumlah_data

'286,113'

In [27]:
# 200000 ribu rumah tangga unique, jika merujuk ke slide presentasi maka representatif hingga tingkat kabupaten/kota
# sekarang ambil informasi provinsi (B1R1) dan kabupaten (B1R2) dan group by untuk tiap rumah tangga 
# dari dataset awal sus12_kons_gizi
daerah_gizi = sus12_kons_gizi.groupby('URUT')[['B1R1','B1R2']].max().reset_index()

# Buat identifikasi untuk daerah tiap RT
daerah_gizi=daerah_gizi.assign(kode_kabupaten=daerah_gizi['B1R1']*100+daerah_gizi['B1R2'])\
            .rename(columns={'B1R1':'kode_prov'}).drop(['B1R2'], axis=1)

# gabung ke data akg harian
gizi_grouped_daily_prov = gizi_grouped_daily.merge(daerah_gizi, left_on='URUT', right_on='URUT')
gizi_grouped_daily_prov.head()

,URUT,kalori,protein,lemak,karbohidrat,WERT,WEIND,jumlah_art,kode_prov,kode_kabupaten
0,110112500101601,1277.633929,37.238643,22.204750,216.257571,183.505020,45.876255,4,11,1101
1,110112500101602,1793.693929,49.248857,37.862571,282.291964,221.008743,55.252186,4,11,1101
2,110112500101603,2593.590357,75.092464,69.346250,385.179429,80.025658,20.006414,4,11,1101
3,110112500101604,1430.819048,39.622619,24.161190,252.797857,254.390213,42.398369,6,11,1101
4,110112500101605,1482.632143,45.375000,32.915893,228.803750,173.681061,43.420265,4,11,1101


### B. Ambil data kabupaten kota dari file excel untuk nama kabupaten/kota

#### Tambahkan nama provinsi

In [28]:
# biar lebih lengkap ambil data nama provinsi dan kabupaten kota
## untuk provinsi dari file pov_line di susenas13, untuk kabupaten kota dari layout modul gabungan susenas12
nama_provinsi = pd.read_csv("./susenas13/pov_line.csv", usecols = ['b1r1','nama_prov'])\
                .drop_duplicates().reset_index(drop=True)

# gabung ke data akg harian
gizi_grouped_daily_prov_name = gizi_grouped_daily_prov.merge(nama_provinsi, left_on='kode_prov', right_on='b1r1')
gizi_grouped_daily_prov_name.head()

,URUT,kalori,protein,lemak,karbohidrat,WERT,WEIND,jumlah_art,kode_prov,kode_kabupaten,b1r1,nama_prov
0,110112500101601,1277.633929,37.238643,22.204750,216.257571,183.505020,45.876255,4,11,1101,11,Aceh
1,110112500101602,1793.693929,49.248857,37.862571,282.291964,221.008743,55.252186,4,11,1101,11,Aceh
2,110112500101603,2593.590357,75.092464,69.346250,385.179429,80.025658,20.006414,4,11,1101,11,Aceh
3,110112500101604,1430.819048,39.622619,24.161190,252.797857,254.390213,42.398369,6,11,1101,11,Aceh
4,110112500101605,1482.632143,45.375000,32.915893,228.803750,173.681061,43.420265,4,11,1101,11,Aceh


#### Tambahkan nama kabupaten kota

In [29]:
## ambil data nama kabupaten kota dari file layout susesnas , sheet label
nama_kabupatenkota_raw = pd.read_excel("./susenas12/Layout -- Susenas 2012 Modul Gabungan.xls", sheet_name='LABEL')
## gunakan kolom value sebagai index , dengan cara fillna dan ambil yang valuenya =  KABU
nama_kabupatenkota_raw['Value'] = nama_kabupatenkota_raw['Value'].fillna(method='ffill')
nama_kabupatenkota = nama_kabupatenkota_raw.set_index('Value').loc['KABU']

# cleansing nama kabupaten kotanya agar lebih rapi dengan remove angka di depan nama kabupaten/kota
nama_kabupatenkota['Label'] = nama_kabupatenkota['Label'].apply(lambda x: x.split(".")[1].strip())
nama_kabupatenkota.head()

# gabung ke data akg harian
gizi_grouped_daily_daerah = gizi_grouped_daily_prov_name.merge(nama_kabupatenkota, 
                                                               left_on='kode_kabupaten', right_on='KODE')
gizi_grouped_daily_daerah = gizi_grouped_daily_daerah.drop(columns=['kode_prov','kode_kabupaten','b1r1','KODE'])
gizi_grouped_daily_daerah.tail()

,URUT,kalori,protein,lemak,karbohidrat,WERT,WEIND,jumlah_art,nama_prov,Label
286108,947122001103806,1381.880000,58.206514,51.578286,178.733429,2190.646820,438.129364,5,Papua,Jayapura
286109,947122001103807,1970.742286,90.243714,73.338057,205.930114,1605.079956,321.015991,5,Papua,Jayapura
286110,947122001103808,2165.612857,98.327143,71.332286,280.153143,649.540222,324.770111,2,Papua,Jayapura
286111,947122001103809,1765.710204,52.837755,44.304082,277.946531,98.785879,14.112268,7,Papua,Jayapura
286112,947122001103810,1573.511905,49.879524,34.153810,248.884048,36.000000,6.000000,6,Papua,Jayapura


### C. Lakukan Aggregasi ke tingkat provinsi --> kabupaten/kota

#### Aggregasi tingkat provinsi 

Pertama kita akan melakukan aggregasi ke tingkat provinsi untuk mengetahui bagaimana distribusi AKG di tiap daerah

In [39]:
# Creating Lambda Function for Weighted Average and Weighted Sum
wt_mean = lambda x: np.average(x, weights=gizi_grouped_daily_daerah.loc[x.index, "WERT"])
wt_sum = lambda x: np.sum(x, weights=gizi_grouped_daily_daerah.loc[x.index, "WERT"])


gizi_provinsi = gizi_grouped_daily_daerah.groupby('nama_prov').agg(kalori = ('kalori', wt_mean),
                                                     protein = ('protein', wt_mean),
                                                     lemak = ('lemak', wt_mean),
                                                     karbohidrat = ('karbohidrat', wt_mean),
#                                                      WEIND = ('WEIND', 'mean')
                                                     WERT = ('WERT', 'sum'))
gizi_provinsi

,kalori,protein,lemak,karbohidrat,WERT
nama_prov,,,,,
Aceh,1860.684230,53.107266,40.606383,294.473860,4.715497e+06
Bali,2034.548725,60.271331,47.326485,315.157327,4.069472e+06
Bangka Belitung,1783.411711,54.574750,39.940991,274.139673,1.302880e+06
Banten,1952.996010,58.715155,46.878960,289.858351,1.124157e+07
Bengkulu,1859.661702,51.543809,41.186873,293.716433,1.772938e+06
DI Yogyakarta,1849.353086,54.419292,47.531534,281.049193,3.530860e+06
DKI Jakarta,1922.298070,61.751251,61.607270,238.655877,9.881391e+06
Gorontalo,1752.228858,48.025248,37.722581,275.082731,1.087665e+06
Jambi,1857.249074,51.302046,42.854140,281.212475,3.254562e+06


##### 5 provinsi top kalori, protein, lemak, karbo tertinggi 

In [83]:
def top_five_akg(dataset,akg : str):
    urut_data = dataset.sort_values(by = akg, ascending = False)
    top_five = urut_data.head()
    return top_five

In [84]:
top_five_akg(gizi_provinsi,'kalori')

,kalori,protein,lemak,karbohidrat,WERT
nama_prov,,,,,
Nusa Tenggara Barat,2036.879448,58.624199,39.149692,338.312634,4.601531e+06
Bali,2034.548725,60.271331,47.326485,315.157327,4.069472e+06
Kalimantan Selatan,1980.835020,57.951040,48.295090,296.552238,3.770838e+06
Sumatera Barat,1965.409867,53.267926,48.569045,294.613193,4.986544e+06
Banten,1952.996010,58.715155,46.878960,289.858351,1.124157e+07


#### Aggregasi tingkat kabupaten/kota

kemudian kita aggregasi ke tingkat kabupaten/kota

In [71]:
# Creating Lambda Function for Weighted Average and Weighted Sum
wt_mean = lambda x: np.average(x, weights=gizi_grouped_daily_daerah.loc[x.index, "WERT"])
wt_sum = lambda x: np.sum(x, weights=gizi_grouped_daily_daerah.loc[x.index, "WERT"])


gizi_kabkota = gizi_grouped_daily_daerah.groupby(['nama_prov','Label']).agg(kalori = ('kalori', wt_mean),
                                                     protein = ('protein', wt_mean),
                                                     lemak = ('lemak', wt_mean),
                                                     karbohidrat = ('karbohidrat', wt_mean),
#                                                      WEIND = ('WEIND', 'mean')
                                                     WERT = ('WERT', 'sum')).reset_index('nama_prov')
gizi_kabkota

,nama_prov,kalori,protein,lemak,karbohidrat,WERT
Label,,,,,,
Aceh Barat,Aceh,1686.216718,49.112493,45.390911,242.917228,181770.125938
Aceh Barat Daya,Aceh,1885.099198,49.736744,35.948125,306.595904,132927.222007
Aceh Besar,Aceh,1812.548884,53.671842,42.324263,278.622117,368743.028731
Aceh Jaya,Aceh,2156.433098,63.780067,52.594420,325.503570,82962.881689
Aceh Selatan,Aceh,1970.410176,52.343946,42.844708,316.200031,212355.504311
...,...,...,...,...,...,...
Tapanuli Selatan,Sumatera Utara,2228.177135,57.926773,44.142485,371.239144,265974.457187
Tapanuli Tengah,Sumatera Utara,1796.487814,51.532546,34.151353,296.197249,324285.717659
Tapanuli Utara,Sumatera Utara,1993.374662,56.724811,34.872185,333.451640,282493.850134


### D. Cek berdasarkan klasifikasi desa/kota

Cek harga pangan untuk tiap daerah : dari amount rupiah dibagi amount kilonya (cek ke data excel) kemudian dipetakan
Cek juga engel's law : pengeluaran untuk makanan apakah proporsinya berkurang dengan naikknya pendapatan

# 3. Distribusi individu (usia, jenis kelamin,tingkat pendidikan)

# 4. Analisis item makanan/minuman penyumbang AKG terbesar 

# 5. Konsumsi air (ml/hari/kapita)

# 6. Korelasi antar indikator gizi

# 7. Korelasi dengan variabel lain

# 8. Data panel ke 2013 ??